# Ia. Inputs Part 1: Define Your Search Terms
* Use this to define the parameters of your Trulia search terms (i.e. min bedroom, max rent)
* Also used to define other search priorities (in our family's case, it was nearby trailheads and parks

In [100]:
# bed and bath are minimums, I'm not sure there's a way to set a maximum
bed = 2
bath = 1
min_sq_ft = 800
max_sq_ft = 2500
min_rent = 1000
max_rent = 8000
home_type = 'SINGLE-FAMILY_HOME_type'

# These are the corners of the latitude/longitude listing search on Trulia
search_lat_long = '37.39164,37.60274,-122.43246,-122.11111'

# What is the latitude/longitude of your work location
# Note it would be cool to set this up to optimize distance between two work locations for dual-income households
office_lat = 37.484948922085856
office_lng = -122.17627131531376

# Nearby Features to look for:
# Note: I used "TH" and "PARK" to help us evaluate homes for the quantity/quality of nearby parks and
# trailheads using Google Maps searches. It would take a bit of finagling but you could customize this
# for anything else you'd want to search for on Google (fitness center, pub, library, cinema, etc.)

# Note 2: This relies on the google maps API and I haven't actuall integrated this query into the functioning piece of this program
th_type = 'park'
th_keyword = 'trailhead'
th_range = 3000
# th_range = what range will you consider for nearby trailheads? (IIRC it's a radius in meters)

park_type = 'park'
park_range = 1000
# park_range = what range will you consider for nearby parks (IIRC it's a radius in meters)


# Ib. Inputs Part 2
These ones are less frequently changed

In [4]:
# Note these are dummy keys--you'll need to replace them with your own
# API Keys
Walk_API_KEY = 'ffd1c56f9a4cc2dfcf31'
# Walk API = the api for WalkScore/BikeScore
google_api_key = 'AIzaSyCklaQEc-njyK0DF2wYHZbs'
# Google API = pulls in commute data and nearby location data

# Other Variable Fields (I dont remember exactly what these are for)
b_mode = 'bicycling'
w_mode = 'walking'

# this arrival time was to predict traffic on Google Maps API Key based on a specific arrival
# time. I picked a random workday in the future to project what my typical commute time might be
arrival_time = 1610553600 # Note: arrival time is 9 AM, Wed, Jan 13, 2021

# Example Listing
# listing_url = 'https://www.trulia.com/p/ca/menlo-park/1165-bieber-ave-menlo-park-ca-94025--2082680255'

# II. Import Libraries

In [5]:
# Universal Parameters
earth_rad = 6373.0

import pickle

import re
import urllib.request
import urllib.parse
import urllib.error
from bs4 import BeautifulSoup
import ssl
import json
import ast
import os
import pandas as pd
import time
import json
from datetime import datetime
from urllib.request import Request, urlopen
from math import sin, cos, sqrt, atan2, radians

# For ignoring SSL certificate errors

ctx = ssl.create_default_context()
ctx.check_hostname = False
ctx.verify_mode = ssl.CERT_NONE

# Lot Size Data -- this was WIP at some point
# https://www.countyoffice.org/property-records-search/?q=43716+Abeloe+Terrace%2C+Fremont%2C+California%2C+USA

# III. Define Classes
* I think that's what these are called
* Honestly I think this is an inefficient use of Classes and I think I saw a whole video about how this is inefficient, but oh well

In [1]:
# class for data retention--this has not been incorporated
class DataStorage():
    def __init__(self, param):
        self.param = param
def save_object(obj):
    try:
        with open("data.pickle", "wb") as f:
            pickle.dump(obj, f, protocol=pickle.HIGHEST_PROTOCOL)
    except Exception as ex:
        print("Error during pickling object (Possibly unsupported):", ex)
def load_object(filename):
    try:
        with open(filename, "rb") as f:
            return pickle.load(f)
    except Exception as ex:
        print("Error during unpickling object (Possibly unsupported):", ex)

In [93]:
# class to pull in all details from Trulia

class TRULIA_LISTING_DETAILS:
    def __init__(self,listing_url):
        start_ts = datetime.now(tz=None)
        self.date = (start_ts.date())
        self.soup = BeautifulSoup(urlopen(Request(listing_url, headers={'User-Agent': 'Mozilla/5.0'})).read(), 'html.parser')
        self.full_address = 0
        self.formatted_address2 = 0
        self.zipcd = 0
        self.list_lat = 0
        self.list_lng = 0
        self.street_address = 0
        self.state = 0
        self.county = 0
        self.city = 0
        self.neighborhood = 0
        self.list_photos = 0
        self.bed = 0
        self.bath = 0
        self.sqft = 0
        self.description = 0
        self.furnished = 0
        self.isActiveForRent = 0
        self.price = 0
        self.dateListed = 0
        self.priceChange = 0
        self.amenities = 0
        self.USD = 0
        self.nbhd = 0
        self.crime = 0
        self.schools = 0
        self.hs_name = 0
        self.hs_rating = 0
        self.hs_max_rating = 0
        self.ms_name = 0
        self.ms_rating = 0
        self.ms_max_rating = 0
        self.es_name = 0
        self.es_rating = 0
        self.es_max_rating = 0

        try:
            json1 = self.soup.find('script', attrs = {'id': "__NEXT_DATA__", 'type':"application/json"}).contents
            js = json.loads(json1[0])
            props = js['props']['_page']
            self.home_details = js['props']['homeDetails']
            self.full_address = js['props']['homeDetails']['location']['homeFormattedAddress']
            self.formatted_address1 = self.full_address.replace(' ', '+')
            self.formatted_address2 = self.formatted_address1.replace(',', '%2C')

    #     LOCATION DETAILS
            try:
                self.zipcd = js['props']['_page']['tracking']['listingZip']
                self.list_lat = js['props']['_page']['tracking']['locationLat']
                self.list_lng = js['props']['_page']['tracking']['locationLon']

                self.street_address = js['props']['homeDetails']['location']['summaryFormattedLocation']
                self.state = js['props']['_page']['tracking']['listingState']
                try:
                    self.county = js['props']['_page']['tracking']['listingCounty']
                except:
                    self.count = 'unknown'
                self.city = js['props']['_page']['tracking']['listingCity']
                try:
                    self.neighborhood = js['props']['_page']['tracking']['listingNeighborhood']
                except:
                    dummy=0
                try:
                    self.list_photos = int(re.findall("See ([0-9]+) photos",self.description)[0])
                except:
                    self.list_photos = None
            except Exception as ex:
                print("LOCATION DETAIL ERROR:", ex)
    # HOUSE DETAILS
            try:
                self.bed = (re.findall("[0-9]+",js['props']['homeDetails']['bedrooms']['branchBannerBedrooms']))[0]
                self.bath = (re.findall("[0-9.]+",js['props']['homeDetails']['bathrooms']['branchBannerBathrooms']))[0]
                self.sqft = int((re.findall("[0-9.,]+",js['props']['homeDetails']['floorSpace']['formattedDimension']))[0].replace(',', ''))
                self.description = js['props']['homeDetails']['description']['value']
        #     Furnished details
                self.heroTags = js['props']['homeDetails']['heroTags']
                # full_address
                for tag in self.heroTags:
                    if tag.get('formattedName')=='FURNISHED':
                        self.furnished=1
                    else:
                        continue
            except Exception as ex:
                print("HOUSE DETAIL ERROR:", ex)

    # LISTING DETAILS
            try:
                self.isActiveForRent = js['props']['homeDetails']['currentStatus']['isActiveForRent']
                if self.isActiveForRent:
                    try:
                        print('Available')
                        self.price = js['props']['homeDetails']['price']['price']
                        try:
                            self.dateListed = js['props']['homeDetails']['activeForRentListing']['dateListed']
                        except:
                            dummy=0
                        try:
                            priceChange1 = js['props']['homeDetails']['priceChange']
                            priceChangeAmt = priceChange1['priceChangeValue']['formattedPrice']
                            priceChangeAmt1 = int(re.findall("([0-9]+)",priceChangeAmt)[0])
                #                 print(priceChangeAmt1)
                            if priceChangeDir=='UP':
                                priceChange = priceChangeAmt1
                            elif priceChangeDir=='DOWN':
                                priceChange = priceChangeAmt1*-1
                            else:
                                self.priceChange = 0
                        except:
                            self.priceChange = 0
                        try:
                            self.typename = js['props']['homeDetails']['price']['__typename']
                            self.amenities = typename['amenities'].get('amenitiesSummaryDescriptionLines')
                        except:
                            dummy=0
                    except:
                        print("LISTING DETAIL ERROR2")
                else:
                    print('Unavailable')
            except Exception as ex:
                print("LISTING DETAIL ERROR:", ex)

    #     Neighborhood Details
            try:
                try:
                    self.nbhd = js['props']['homeDetails']['surroundings']['localUGC']['stats']['attributes']
                except:
                    self.nbhd = "N/A"
                try:
                    self.crime = js['props']['homeDetails']['localInfoSummary']['crime'].get('crimeSummaryDescription').split(' ', 1)[0]
                except:
                    self.crime = "N/A"
                try:
                    self.schools = js['props']['homeDetails']['assignedSchools']['schools']
                    for school in self.schools:
                        category2 = None
                        name = school.get('name')
                        category = school.get('categories')[0]
                        try:
                            category2 = school.get('categories')[1]
                        except:
                            data = None
                        enrollment = school.get('enrollmentType')
                        providerRating = school.get('providerRating')

                        rating = providerRating.get('rating')
                        max_rating = providerRating.get('maxRating')
                        if enrollment == 'PUBLIC':
                            if category == 'HIGH' or category2 =='HIGH':
                                self.hs_name = name
                                self.hs_rating = rating
                                self.hs_max_rating = max_rating
                            else:
                                data = None
                            if category == 'MIDDLE' or category2 =='MIDDLE':
                                self.ms_name = name
                                self.ms_rating = rating
                                self.ms_max_rating = max_rating
                            else:
                                data = None
                            if category == 'ELEMENTARY' or category2 =='ELEMENTARY':
                                self.es_name = name
                                self.es_rating = rating
                                self.es_max_rating = max_rating
                            else:
                                data = None
                        else:
                            data = None
                except:
                    dummy = 0

            except:
                print('Neighborhood Detail Error')
            try:
                self.json_url_data = (listing_url, self.date, self.full_address, self.formatted_address2, self.zipcd, self.list_lat, self.list_lng, self.street_address, self.state, self.county, self.city, self.neighborhood, self.list_photos, self.bed, self.bath, self.sqft, self.description, self.furnished, self.isActiveForRent, self.price, self.dateListed, self.priceChange, self.amenities, self.nbhd, self.crime, self.schools, self.hs_name, self.hs_rating, self.hs_max_rating, self.ms_name, self.ms_rating, self.ms_max_rating, self.es_name, self.es_rating, self.es_max_rating)
                json_url_data = self.json_url_data
            except Exception as ex:
                print("JSON Error:", ex)
            print('Instance Created')
        except Exception as ex:
            print("Core Error:", ex)

    def print_url_data(json_url_data):
        try:
            print(json_url_data)
        except Exception as ex:
            print("Error:", ex)

In [65]:
# Class to pull in all detail from WalkScore (Bike/Transit scores included)

class WALKSCORE_DETAILS:
    def __init__(self,trulia_data):
        start_ts = datetime.now(tz=None)
        self.walk_url = f"https://api.walkscore.com/score?format=json&lat={trulia_data.list_lat}&lon={trulia_data.list_lng}&transit=1&bike=1&wsapikey={Walk_API_KEY}"
        uh = urllib.request.urlopen(self.walk_url, context=ctx)
        data = uh.read().decode()
        try:
            js_w = json.loads(data)
        except:
            print(data)  # We print in case unicode causes an error

        #     results = js["results"]
        #     print(js)
        try:
            self.walk = js_w['walkscore']
        except:
            self.walk = None
        try:
            self.transit = js_w['transit']['score']
        except:
            self.transit = None
        try:
            self.bike = js_w['bike']['score']
        except:
            self.bike = None
    def PRINT_WALKSCORE_DETAILS(json_url_data):
        try:
            print(json_url_data)
        except Exception as ex:
            print("Error:", ex)

In [90]:
# Class to pull in details on travel distance between two locations
# Note that Google Maps data is the only data that costs money (assuming your free trial has expired),
# so data retention/optimization is more important here. The cost comes out to $0.005 per hit, but I
# think the way this query is set up each time you call it counts as 6 hits?
# When I was working on this project in earnest, I spend $200 worth of free trial credits running this
# API for commute, trailhead data (see code excerpts below)
# Given that I was still under a free trial at that time, I didn't really care and you may not either

class GOOGLE_MAPS_TRAVEL_DETAILS:
    def __init__(self,lat1,lng1,lat2,lng2):
        start_ts = datetime.now(tz=None)
        commute_url = f"https://maps.googleapis.com/maps/api/distancematrix/json?origins={lat1},{lng1}|{lat2},{lng2}&destinations={lat1},{lng1}|{lat2},{lng2}&arrival_time={arrival_time}&mode={b_mode}&key={google_api_key}"
        try:
            uh = urllib.request.urlopen(commute_url, context=ctx)
            data = uh.read().decode()
            google_json = json.loads(data)
            self.google_json = google_json
            print("Google Maps API Data Acquired")
        except Exception as ex:
            print("Error:", ex)
#             Note that this "GOOGLE_MAPS_EXTRACTION" keeps erroring out so I'm just pulling the detail here manually
    def GOOGLE_MAPS_EXTRACTION(self,input_data):
        try:
            print(input_data)
        except Exception as ex:
            print("Error:", ex)

# IV.a. Getting a list of URLs
* This code pulls in all of the listings under the search criteria defined above and then puts them into a list. It also saves it to a CSV file so that you don't have to repeatedly run this query
* Note: This seems overly complicated, the reason for all of this is in the case where the search results extent on to 2+ pages, this goes through each of those pages to find all of the URLs

In [ ]:
# listings_url_links = a list of all of the urls for specific home listings
listings_url_links = list()

listings = list()

listings_url = f"https://www.trulia.com/for_rent/{search_lat_long}_xy/{bed}p_beds/{bath}p_baths/{min_rent}-{max_rent}_price/{min_sq_ft}-{max_sq_ft}_sqft/{home_type}/11_zm/"

# This query runs through all of the pages of results on Trulia

while listings_url not in listings_url_links:
    print(listings_url)
    listings_url_links.append(listings_url)
    req = Request(listings_url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()
    links = list()

    soup = BeautifulSoup(webpage, 'html.parser')
    html = soup.prettify('utf-8')
    test = soup.body
    results = test.find("ul", {"data-testid": "search-result-list-container"})
    for a in results.find_all('a', href=True):
        append = a['href']
        follow = 'https://www.trulia.com' + append
        if follow not in listings:
            listings.append(follow)
    try:
        next = test.find("a", {"rel": "next"})
        nextlink = next['href']
        listings_url = 'https://www.trulia.com'+nextlink
        time.sleep(5)
#     Note: sleep is to keep the Trulia page from blocking me--when I was trying to pull in lots of data I had this at a sleep timer of 60 seconds
        continue
    except:
        print('Listings compiled')
        print(listings_url_links)

In [102]:
listing_urls = pd.DataFrame(listings, columns =['url'])
listing_urls.to_csv('listing_urls.csv')
print("to_csv complete")

to_csv complete


# IV.b. A list of URLs to use for practice
Use this as an override while working with the data so that I'm only pulling a small sample instead of 50+ listings

In [91]:
listings = (
    'https://www.trulia.com/p/ca/redwood-city/address-not-disclosed-redwood-city-ca-94061--2208275442',
    'https://www.trulia.com/p/ca/millbrae/484-anita-dr-millbrae-ca-94030--2082701788',
)

for link in listings:
    print(listings)
# len(listings)
print(len(listings))

('https://www.trulia.com/p/ca/redwood-city/address-not-disclosed-redwood-city-ca-94061--2208275442', 'https://www.trulia.com/p/ca/millbrae/484-anita-dr-millbrae-ca-94030--2082701788')
('https://www.trulia.com/p/ca/redwood-city/address-not-disclosed-redwood-city-ca-94061--2208275442', 'https://www.trulia.com/p/ca/millbrae/484-anita-dr-millbrae-ca-94030--2082701788')
2


# V. The magical code that gets all of the data for each listing in your dataset
This is incomplete but I decided to leave out the trailhead/park/school/neighborhood data for simplicity. You can see the code excerpts below this if you're interested in incorporating some of that back into this query

In [94]:
listing_outputs = list()
i = 0
total_rows = len(listings)
ts1 = datetime.now(tz=None)
init_ts = ts1
# print(listing_details)
# Making the website believe that you are accessing it using a mozilla browser
json_data = list()

for listing_url in listings:
    i += 1
    start_ts = datetime.now(tz=None)
    date = (start_ts.date())
    print(i," of ",total_rows," - ",start_ts.time(), listing_url)
    trulia_data = TRULIA_LISTING_DETAILS(listing_url)
    walkscore_data = WALKSCORE_DETAILS(trulia_data)
#     commute_details = GOOGLE_MAPS_TRAVEL_DETAILS(trulia_data.list_lat,trulia_data.list_lng,office_lat,office_lng)
#     elements = commute_details.google_json['rows'][0].values()
#     elements2 = commute_details.google_json['rows'][1].values()

#     for val in elements:
#         el = val[1]
#         dist = {key: el[key] for key in el.keys() & {'distance'}} 
#         dist2 = dist.values()
#         for val in dist2:
#             val2 = val.values()
#             for alpha in val2:
#                 try:
#                     travel_dist_to = int(alpha)
#                 except:
#                     travel_dist_to = None
#                     continue
#         dur = {key: el[key] for key in el.keys() & {'duration'}} 
#         dur2 = dur.values()
#         for val in dur2:
#             val2 = val.values()
#             for alpha in val2:
#                 try:
#                     travel_time_to = int(alpha)
#                 except:
#                     travel_time_to = None
#                     continue

#     for val in elements2:
#         el = val[0]
#         dist = {key: el[key] for key in el.keys() & {'distance'}} 
#         dist2 = dist.values()
#         for val in dist2:
#             val2 = val.values()
#             for alpha in val2:
#                 try:
#                     travel_dist_from = int(alpha)
#                 except:
#                     travel_dist_from = None
#                     continue
#         dur = {key: el[key] for key in el.keys() & {'duration'}} 
#         dur2 = dur.values()
#         for val in dur2:
#             val2 = val.values()
#             for alpha in val2:
#                 try:
#                     travel_time_from = int(alpha)
#                 except:
#                     travel_time_from = None
#                     continue
    listing_rowdata = (trulia_data.date, trulia_data.dateListed, trulia_data.description, trulia_data.full_address, trulia_data.formatted_address2, trulia_data.street_address, trulia_data.state, trulia_data.county, trulia_data.city, trulia_data.neighborhood, trulia_data.zipcd, trulia_data.list_photos, trulia_data.furnished, trulia_data.list_lat, trulia_data.list_lng, trulia_data.price, trulia_data.typename, trulia_data.priceChange, trulia_data.bed, trulia_data.bath, trulia_data.sqft, walkscore_data.walk, walkscore_data.bike, walkscore_data.transit)
    if listing_rowdata not in listing_outputs:
        listing_outputs.append(listing_rowdata)
        
#     travel_dist_to, travel_time_to, travel_dist_from, travel_time_from, 
#                    Sidewalks, KidsPlayOutside, Wildlife, FriendlyNeighbors, GroceryWalk, LongTermResidents, WalkAloneNight, HolidaySpirit, listing_url, cover_image_url, listingID, maloneID, trailheads, parks, closest_park, closest_park_id, closest_park_rating, closest_park_ratings, closest_park_distance, most_popular_park, most_popular_park_id, most_popular_park_rating, most_popular_park_ratings, most_popular_park_distance, closest_trailhead, closest_trailhead_id, closest_trailhead_rating, closest_trailhead_ratings, closest_trailhead_distance, most_popular_trailhead, most_popular_trailhead_id, most_popular_trailhead_rating, most_popular_trailhead_ratings, most_popular_trailhead_distance, es_name, es_rating, es_max_rating, ms_name, ms_rating, ms_max_rating, hs_name, hs_rating, hs_max_rating, crime, Restaurants, Groceries, Nightlife)


1  of  2  -  22:45:49.941610 https://www.trulia.com/p/ca/redwood-city/address-not-disclosed-redwood-city-ca-94061--2208275442
Available
Instance Created
2  of  2  -  22:45:50.840259 https://www.trulia.com/p/ca/millbrae/484-anita-dr-millbrae-ca-94030--2082701788
Available
Instance Created


# VI. Write listing data to CSV file
I then uploaded this to Tableau, where it was easier to play around with visualization, develop a scoring methodology, and iterate

In [97]:
listing_details_csv = pd.DataFrame(listing_outputs, columns =['date','dateListed','description', 'full_address', 'formatted_address2', 'street_address', 'state', 'county', 'city', 'neighborhood', 'zipcd', 'list_photos', 'furnished','lat', 'lng', 'price', 'typename', 'priceChange', 'bed', 'bath', 'sqft', 'walk', 'bike', 'transit'])
listing_details_csv.to_csv('listing_details_csv.csv')
print("Complete")

Complete


# Appendix: Old Code, including some really useful stuff
The code as I left it was in a bit of a messy state, so I've tried to deliver something that was almost immediately usable to you. However, that means that several potentially useful pieces are excluded. I'm annotating this a little bit, but mainly we can chat about this if you have any questions. Feel free to find me on FB (https://www.facebook.com/profile.php?id=17830791) and ping me on Messenger

## Import latest version of listing_details_csv
Some version of this was to pull in details from a CSV instead of re-running a query

In [13]:
listing_details_df = pd.read_csv('listing_details_csv_draft.csv')
listing_details_df = listing_details_df.drop(listing_details_df.columns[0], axis =1)
listing_details_df

listing_details_url = listing_details_df['listing_url']
listing_details_url

listing_urls = pd.read_csv('listing_urls.csv')
listing_urls = listing_urls.drop(listing_urls.columns[0], axis =1)
listing_urls
listing_urls_urls = listing_urls['url']
listing_urls_urls =listing_urls.values.tolist()
listings = list()
for link in listing_urls_urls:
    listings.append(link[0])

print(len(listings))
    
# for link in listing_urls:
#     listing_urls_urls.append(link)
    

# # listing_urls_urls
# print(listing_urls)
# listings
# for 


27


## Original Primary Code
### I think this is the code exactly as I left it, including some notes here:
Changes to make:
* Add to table without overwriting (allows us to expand to other areas without starting over)

Changes to Validate:
* Update with a "Projected Completion Time"
* Add in neighborhood descriptions (groceries, restaurants, nightlife)
* Added in a "skip" if listing is no longer available for rent
* Added in comments on certain error types
* Lot size

Dataset 3.0 Goals:
* Price change history
* Connect with Great Schools API, build database for schools that can be linked rather than repeatedly calling it
* Crime Data - connect with Crimeometer
* Add weight to most popular parks/trailhead into model - See Tableau use for approach
* Build score into the python file instead of Tableau (this requires the model to be somewhat finalized)
* Maybe more on school?
* Parks/trailheads need refinement
* Library distance
* Traffic?

Dataset 4.0 goals
* Climate data - see Weatherspark for inspiration.

Deprioritized:
* Automated email report on a regular basis of new listings with scores, links, etc.

Prior Changes
* Change empty to Not a Number - COMPLETE
* Number of photographs in listing (proxy for quality) - COMPLETE
* Number of ratings for different things (trailheads, parks) - COMPLETE
* Clean up text "sqft", etc. - COMPLETE
* Add in a date thing - COMPLETE
* Add in crime - in progress - COMPLETE
* Output to csv every 50 or so? - COMPLETE

In [5]:
listing_details = list()
i = 0
total_rows = len(listings)
ts1 = datetime.now(tz=None)
init_ts = ts1
# print(listing_details)
# Making the website believe that you are accessing it using a mozilla browser
for link in listings:
    listing_url = link
    i = i+1
    start_ts = datetime.now(tz=None)
    date = (start_ts.date())
    print(i," of ",total_rows," - ",start_ts.time(), listing_url)
    
    trailheads = list()
    parks = list()
    hs_name = None
    ms_name = None
    es_name = None
    hs_rating = None
    ms_rating = None
    es_rating = None
    hs_max_rating = None
    ms_max_rating = None
    es_max_rating = None
    furnished = 0

    req = Request(listing_url, headers={'User-Agent': 'Mozilla/5.0'})
    webpage = urlopen(req).read()

    soup = BeautifulSoup(webpage, 'html.parser')

    try:
        json1 = soup.find('script', attrs = {'id': "__NEXT_DATA__", 'type':"application/json"}).contents
#         print(json1)
        js = json.loads(json1[0])
#         print(js)
        isActiveForRent = js['props']['homeDetails']['currentStatus']['isActiveForRent']
        if isActiveForRent:
            print('Available')
            print(js['props']['homeDetails'])
            full_address = js['props']['homeDetails']['location']['homeFormattedAddress']
            formatted_address1 = full_address.replace(' ', '+')
            formatted_address2 = formatted_address1.replace(',', '%2C')
            
#             sqft = (re.findall("[0-9.,]+",js['props']['homeDetails']['floorSpace']['formattedDimension']))[0].replace(',', '')

            street_address = js['props']['homeDetails']['location']['summaryFormattedLocation']
            description = js['props']['homeDetails']['description']['value']
            try:
                cs_req = Request(lot_size_url, headers={'User-Agent': 'Mozilla/5.0'})
                cs_webpage = urlopen(cs_req).read()
                cs_body = BeautifulSoup(cs_webpage, 'html.parser').body
                LotSize1 = (cs_body.find('th', text=re.compile('Area')).nextSibling).contents[0][0:6].rstrip()
                LotSize = int(LotSize1.replace(',', ''))
            except:
                LotSize = None
            try:
                list_photos = int(re.findall("See ([0-9]+) photos",description)[0])
            except:
                list_photos = None
            try:
                cover_image_url = js['props']['_page']['metaTags']['ogImage']['content']
            except:
                cover_image_url = None
            try:
                dateListed = js['props']['homeDetails']['activeForRentListing']['dateListed']
            except:
                dateListed = 'unknown'
            heroTags = js['props']['homeDetails']['heroTags']
            # full_address
            for tag in heroTags:
                if tag.get('formattedName')=='FURNISHED':
                    furnished=1
                else:
                    continue

            print(full_address)
            # href = js['props']['_page']['href']
            schools = js['props']['homeDetails']['assignedSchools']['schools']

            for school in schools:
                category2 = None
                name = school.get('name')
                category = school.get('categories')[0]
                try:
                    category2 = school.get('categories')[1]
                except:
                    data = None
                enrollment = school.get('enrollmentType')
                providerRating = school.get('providerRating')

                rating = providerRating.get('rating')
                max_rating = providerRating.get('maxRating')
                if enrollment == 'PUBLIC':
                    if category == 'HIGH' or category2 =='HIGH':
                        hs_name = name
                        hs_rating = rating
                        hs_max_rating = max_rating
                    else:
                        data = None
                    if category == 'MIDDLE' or category2 =='MIDDLE':
                        ms_name = name
                        ms_rating = rating
                        ms_max_rating = max_rating
                    else:
                        data = None
                    if category == 'ELEMENTARY' or category2 =='ELEMENTARY':
                        es_name = name
                        es_rating = rating
                        es_max_rating = max_rating
                    else:
                        data = None
                else:
                    data = None
    #         print(es_name, es_rating, es_max_rating, ms_name, ms_rating, ms_max_rating, hs_name, hs_rating, hs_max_rating)
            try:
                listingID = js['props']['_page']['tracking']['listingID']
            except:
                listingID = 'None'
            try:
                maloneID = js['props']['_page']['tracking']['maloneID']
            except:
                maloneID = 'None'
            try:
                state = js['props']['_page']['tracking']['listingState']
            except:
                state = 'None'
            try:
                county = js['props']['_page']['tracking']['listingCounty']
            except:
                count = 'unknown'
            try:
                city = js['props']['_page']['tracking']['listingCity']
            except:
                city = 'None'
            try:
                neighborhood = js['props']['_page']['tracking']['listingNeighborhood']
            except:
                neighborhood = 'N/A'
            try:
                typename = js['props']['homeDetails']['localInfoSummary']
                crime = typename['crime'].get('crimeSummaryDescription').split(' ', 1)[0]
            except:
                typename=None
                crime = None
            try:
                amenities = typename['amenities'].get('amenitiesSummaryDescriptionLines')
                Restaurants = int(re.findall("([0-9]+)",amenities)[0])
                Groceries = int(re.findall("([0-9]+)",amenities)[1])
                Nightlife = int(re.findall("([0-9]+)",amenities)[1])
                
            except:
                amenities = None
                Restaurants = None
                Groceries = None
                Nightlife = None

            zipcd = js['props']['_page']['tracking']['listingZip']
            list_lat = js['props']['_page']['tracking']['locationLat']
            list_lng = js['props']['_page']['tracking']['locationLon']
            try:
                price = js['props']['homeDetails']['price']['price']
            except:
                price = None
            try:
                USD = js['props']['homeDetails']['price']['currencyCode']
                typename = js['props']['homeDetails']['price']['__typename']
            except:
                USD = None
            priceChange = 0
            try:
                priceChange1 = js['props']['homeDetails']['priceChange']
                priceChangeAmt = priceChange1['priceChangeValue']['formattedPrice']
                priceChangeAmt1 = int(re.findall("([0-9]+)",priceChangeAmt)[0])
#                 print(priceChangeAmt1)
                if priceChangeDir=='UP':
                    priceChange = priceChangeAmt1
                elif priceChangeDir=='DOWN':
                    priceChange = priceChangeAmt1*-1
                else:
                    priceChange = 0
            except:
                priceChange = 0
            bed = (re.findall("[0-9]+",js['props']['homeDetails']['bedrooms']['branchBannerBedrooms']))[0]
            bath = (re.findall("[0-9.]+",js['props']['homeDetails']['bathrooms']['branchBannerBathrooms']))[0]
            sqft = int((re.findall("[0-9.,]+",js['props']['homeDetails']['floorSpace']['formattedDimension']))[0].replace(',', ''))
            

            try:
                nbhd = js['props']['homeDetails']['surroundings']['localUGC']['stats']['attributes']
                for item in nbhd:
                    sw = 'Sidewalks'
                    kp = 'KidsPlayOutside'
                    wl = 'Wildlife'

                    fn = 'FriendlyNeighbors'
                    wg = 'WalkableToGroceries'
                    lt = 'LongTermResidents'
                    wa = 'PeopleWouldWalkAloneAtNight'
                    hs = 'HolidaySpirit'

                    temp = list(item.values())
                    result = [ele for ele in temp if ele == sw]
                    if sw in result:
                        dur = {key: item[key] for key in item.keys() & {'score'}} 
                        for key, value in dur.items():
                            Sidewalks = value

                    result = [ele for ele in temp if ele == kp]
                    if kp in result:
                        dur = {key: item[key] for key in item.keys() & {'score'}} 
                        for key, value in dur.items():
                            KidsPlayOutside = value

                    result = [ele for ele in temp if ele == wl]
                    if wl in result:
                        dur = {key: item[key] for key in item.keys() & {'score'}} 
                        for key, value in dur.items():
                            Wildlife = value

                    result = [ele for ele in temp if ele == fn]
                    if fn in result:
                        dur = {key: item[key] for key in item.keys() & {'score'}} 
                        for key, value in dur.items():
                            FriendlyNeighbors = value

                    result = [ele for ele in temp if ele == wg]
                    if wg in result:
                        dur = {key: item[key] for key in item.keys() & {'score'}} 
                        for key, value in dur.items():
                            GroceryWalk = value

                    result = [ele for ele in temp if ele == lt]
                    if lt in result:
                        dur = {key: item[key] for key in item.keys() & {'score'}} 
                        for key, value in dur.items():
                            LongTermResidents = value

                    result = [ele for ele in temp if ele == wa]
                    if wa in result:
                        dur = {key: item[key] for key in item.keys() & {'score'}} 
                        for key, value in dur.items():
                            WalkAloneNight = value

                    result = [ele for ele in temp if ele == hs]
                    if hs in result:
                        dur = {key: item[key] for key in item.keys() & {'score'}} 
                        for key, value in dur.items():
                            HolidaySpirit = value
            except:
                Sidewalks = None
                KidsPlayOutside = None
                Wildlife = None
                FriendlyNeighbors = None
                GroceryWalk = None
                LongTermResidents = None
                WalkAloneNight = None
                HolidaySpirit = None

# Get Walk/Bike/TransitScore

            walk_url = f"https://api.walkscore.com/score?format=json&lat={list_lat}&lon={list_lng}&transit=1&bike=1&wsapikey={Walk_API_KEY}"
            uh = urllib.request.urlopen(walk_url, context=ctx)
            data = uh.read().decode()
            try:
                js_w = json.loads(data)
            except:
                print(data)  # We print in case unicode causes an error

            #     results = js["results"]
            #     print(js)
            try:
                walk = js_w['walkscore']
            except:
                walk = None
            try:
                transit = js_w['transit']['score']
            except:
                transit = None
            try:
                bike = js_w['bike']['score']
            except:
                bike = None

# Pull in Commute data from Google API to calculate biking distance time to/from office
            commute_url = f"https://maps.googleapis.com/maps/api/distancematrix/json?origins={list_lat},{list_lng}|{office_lat},{office_lng}&destinations={list_lat},{list_lng}|{office_lat},{office_lng}&arrival_time={a_arrival_time}&mode={b_mode}&key={google_api_key}"
            try:
                uh = urllib.request.urlopen(commute_url, context=ctx)
                data = uh.read().decode()
                js_g = json.loads(data)
                elements = js_g['rows'][0].values()
                elements2 = js_g['rows'][1].values()

                for val in elements:
                    el = val[1]
                    dist = {key: el[key] for key in el.keys() & {'distance'}} 
                    dist2 = dist.values()
                    for val in dist2:
                        val2 = val.values()
                        for alpha in val2:
                            try:
                                travel_dist_to = int(alpha)
                            except:
                                travel_dist_to = None
                                continue
                    dur = {key: el[key] for key in el.keys() & {'duration'}} 
                    dur2 = dur.values()
                    for val in dur2:
                        val2 = val.values()
                        for alpha in val2:
                            try:
                                travel_time_to = int(alpha)
                            except:
                                travel_time_to = None
                                continue

                for val in elements2:
                    el = val[0]
                    dist = {key: el[key] for key in el.keys() & {'distance'}} 
                    dist2 = dist.values()
                    for val in dist2:
                        val2 = val.values()
                        for alpha in val2:
                            try:
                                travel_dist_from = int(alpha)
                            except:
                                travel_dist_from = None
                                continue
                    dur = {key: el[key] for key in el.keys() & {'duration'}} 
                    dur2 = dur.values()
                    for val in dur2:
                        val2 = val.values()
                        for alpha in val2:
                            try:
                                travel_time_from = int(alpha)
                            except:
                                travel_time_from = None
                                continue

            except:
                print("commute_url error")
                print(data)  # We print in case unicode causes an error


# 
# Parks and Trailhead URL data from Google API
# 

            th_url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={list_lat},{list_lng}&radius={th_range}&type={th_type}&keyword={th_keyword}&key={google_api_key}"
# Trailheads Data
            try:
                try:
                    uh = urllib.request.urlopen(th_url, context=ctx)
                    data = uh.read().decode()
                    js = json.loads(data)
                    th_results = js["results"]
                except:
                    print("Trailhead json error")  # We print in case unicode causes an error
                    print(data)  # We print in case unicode causes an error
                for v in th_results:
                    geo = v["geometry"]
                    p_lat = v["geometry"]['location']['lat']
                    p_lng = v["geometry"]['location']['lng']
                    name = v["name"]
                    trailhead_id = v["id"]

                    types = v["types"]
                    try:
                        rating = v["rating"]
                    except:
                        rating = None
                    try:
                        rating_cnt = v["user_ratings_total"]
                    except:
                        rating_cnt = None
                    try:
                        rating_wt = rating*rating_cnt
                    except:
                        rating_wt = None



            # 
            # Pull in Trailhead Distances
            # 

                    trailhead_dist_url = f"https://maps.googleapis.com/maps/api/distancematrix/json?origins={list_lat},{list_lng}&destinations={p_lat},{p_lng}&mode=walking&key={google_api_key}"
                    try:
                        uh = urllib.request.urlopen(trailhead_dist_url, context=ctx)
                        data = uh.read().decode()
                        js_g = json.loads(data)
                    except:
                        print(data)  # We print in case unicode causes an error

                    elements = js_g['rows'][0].values()
                    for val in elements:
                        el = val[0]
                        dist = {key: el[key] for key in el.keys() & {'distance'}} 
                        dist2 = dist.values()
                        for val in dist2:
                            val2 = val.values()
                            for alpha in val2:
                                try:
                                    distance = int(alpha)
                                except:
                                    distance = None
                                    continue
                    th_rowdata = (city, state, list_lat, list_lng, name,p_lat,p_lng,rating, rating_cnt, types, rating_wt, distance, trailhead_id)
                    trailheads.append(th_rowdata)

                trailheads_df = pd.DataFrame(trailheads, columns =['city', 'state', 'lat', 'lng', 'name','p_lat','p_lng','rating', 'rating_cnt', 'types', 'rating_wt', 'distance','trailhead_id'])
                trailheads = len(trailheads_df.index)
    #             print(trailheads_df)

                closest_trailhead_dist = trailheads_df['distance'].min()
                closest_trailhead_dets = trailheads_df.loc[trailheads_df['distance']==closest_trailhead_dist]
                closest_trailhead_list = closest_trailhead_dets.values.tolist()[0]
                closest_trailhead = closest_trailhead_list[4]
                closest_trailhead_id = closest_trailhead_list[12]
                closest_trailhead_rating = closest_trailhead_list[7]
                closest_trailhead_distance = closest_trailhead_list[8]
                closest_trailhead_ratings = closest_trailhead_list[8]
                closest_trailhead_distance = closest_trailhead_list[11]

                most_popular_trailhead_dist = trailheads_df['rating_cnt'].max()
                most_popular_trailhead_dets = trailheads_df.loc[trailheads_df['rating_cnt']==most_popular_trailhead_dist]
                most_popular_trailhead_list = most_popular_trailhead_dets.values.tolist()[0]
                most_popular_trailhead = most_popular_trailhead_list[4]
                most_popular_trailhead_id = most_popular_trailhead_list[12]
                most_popular_trailhead_rating = most_popular_trailhead_list[7]
                most_popular_trailhead_distance = most_popular_trailhead_list[8]
                most_popular_trailhead_ratings = most_popular_trailhead_list[8]
                most_popular_trailhead_distance = most_popular_trailhead_list[11]

            except:
    #             print("No trailheads")
                trailheads = 0
                closest_trailhead_dist = None
                closest_trailhead_dets = None
                closest_trailhead_list = None
                closest_trailhead = None
                closest_trailhead_id = None
                closest_trailhead_rating = None
                closest_trailhead_ratings = None
                closest_trailhead_distance = None

                most_popular_trailhead_dist = None
                most_popular_trailhead_dets = None
                most_popular_trailhead_list = None
                most_popular_trailhead = None
                most_popular_trailhead_id = None
                most_popular_trailhead_rating = None
                most_popular_trailhead_ratings = None
                most_popular_trailhead_distance = None

# 
# Parks Data
# 

            parks_url = f"https://maps.googleapis.com/maps/api/place/nearbysearch/json?location={list_lat},{list_lng}&radius={park_range}&type={park_type}&key={google_api_key}"
            try:
                uh = urllib.request.urlopen(parks_url, context=ctx)
                data = uh.read().decode()
                try:
                    js = json.loads(data)
                except:
                    print(data)  # We print in case unicode causes an error

                results = js["results"]
                for v in results:
                    geo = v["geometry"]
                    p_lat = v["geometry"]['location']['lat']
                    p_lng = v["geometry"]['location']['lng']
                    name = v["name"]
                    park_id = v["id"]
                    types = v["types"]
                    try:
                        rating = v["rating"]
                    except:
                        rating = None
                    try:
                        rating_cnt = v["user_ratings_total"]
                    except:
                        rating_cnt = None
                    try:
                        rating_wt = rating*rating_cnt
                    except:
                        rating_wt = None

                    park_url = f"https://maps.googleapis.com/maps/api/distancematrix/json?origins={list_lat},{list_lng}&destinations={p_lat},{p_lng}&mode=walking&key={G_api_key}"
    #                 print(park_url)
                    uh = urllib.request.urlopen(park_url, context=ctx)
                    data = uh.read().decode()
                    try:
                        js_g = json.loads(data)
                    except:
                        print(data)  # We print in case unicode causes an error

                    elements = js_g['rows'][0].values()
        #             print(elements)
                    for val in elements:
                        el = val[0]
                        dist = {key: el[key] for key in el.keys() & {'distance'}} 
                        dist2 = dist.values()
                        for val in dist2:
                            val2 = val.values()
    #                         print(val2)
                            for alpha in val2:
                                try:
                                    distance = int(alpha)
                                except:
                                    distance = None
                                    continue
                    rowdata = (city, state, list_lat, list_lng, name,p_lat,p_lng,rating, rating_cnt, types, rating_wt, distance, park_id)
        #             print(rowdata)
                    parks.append(rowdata)
                parks_df = pd.DataFrame(parks, columns =['city', 'state', 'lat', 'lng', 'name','p_lat','p_lng','rating', 'rating_cnt', 'types', 'rating_wt', 'distance', 'park_id'])

                parks = len(parks_df.index)

                closest_park_dist = parks_df['distance'].min()
                closest_park_dets = parks_df.loc[parks_df['distance']==closest_park_dist]
                closest_park_list = closest_park_dets.values.tolist()[0]
    #             print(closest_park_list)
                closest_park = closest_park_list[4]
                closest_park_id = closest_park_list[12]
                closest_park_rating = closest_park_list[7]
                closest_park_ratings = closest_park_list[8]
                closest_park_distance = closest_park_list[11]

                most_popular_park_rating = parks_df['rating_cnt'].max()
                most_popular_park_dets = parks_df.loc[parks_df['rating_cnt']==most_popular_park_rating]
                most_popular_park_list = most_popular_park_dets.values.tolist()[0]
    #             print(most_popular_park_list)
                most_popular_park = most_popular_park_list[4]
                most_popular_park_id = most_popular_park_list[12]
                most_popular_park_rating = most_popular_park_list[7]
                most_popular_park_ratings = most_popular_park_list[8]
                most_popular_park_distance = most_popular_park_list[11]

            except:
    #             print("No parks")
                parks = 0

                closest_park_dist = None
                closest_park_dets = None
                closest_park_list = None
                closest_park = None
                closest_park_id = None
                closest_park_rating = None
                closest_park_ratings = None
                closest_park_distance = None


                most_popular_park_rating = None
                most_popular_park_dets = None
                most_popular_park_list = None
                most_popular_park = None
                most_popular_park_id = None
                most_popular_park_rating = None
                most_popular_park_ratings = None
                most_popular_park_distance = None

            listing_rowdata = (date, dateListed, description, full_address, formatted_address2, street_address, state, county, city, neighborhood, zipcd, list_photos, furnished, list_lat, list_lng, price, USD, typename, priceChange, bed, bath, sqft, LotSize, walk, bike, transit, Sidewalks, KidsPlayOutside, Wildlife, FriendlyNeighbors, GroceryWalk, LongTermResidents, WalkAloneNight, HolidaySpirit, travel_dist_to, travel_time_to, travel_dist_from, travel_time_from, listing_url, cover_image_url, listingID, maloneID, trailheads, parks, closest_park, closest_park_id, closest_park_rating, closest_park_ratings, closest_park_distance, most_popular_park, most_popular_park_id, most_popular_park_rating, most_popular_park_ratings, most_popular_park_distance, closest_trailhead, closest_trailhead_id, closest_trailhead_rating, closest_trailhead_ratings, closest_trailhead_distance, most_popular_trailhead, most_popular_trailhead_id, most_popular_trailhead_rating, most_popular_trailhead_ratings, most_popular_trailhead_distance, es_name, es_rating, es_max_rating, ms_name, ms_rating, ms_max_rating, hs_name, hs_rating, hs_max_rating, crime, Restaurants, Groceries, Nightlife)
            if listing_rowdata not in listing_details:
                listing_details.append(listing_rowdata)
            finish_ts = datetime.now(tz=None)
            print("Data Pull Time:",finish_ts-ts1)
            try:
                ts1 = finish_ts
                TotalTime = finish_ts-init_ts
                Progress = total_rows - i
                ProcTim = TotalTime/Progress
                Proj_Finish_TS = ProcTim*(total_rows-i)+datetime.now(tz=None)
                print("Projected Completion TS:",Proj_Finish_TS)
                
                ts1 = finish_ts
                TotalTime = finish_ts-init_ts
                Progress = total_rows - i
                ProcTim = TotalTime/Progress
                Proj_Finish_TS = ProcTim*(total_rows-i)+datetime.now(tz=None)
                print("Projected Completion TS:",Proj_Finish_TS)

            except:
                data = None
        else:
            print('Listing Unavailable')
    except:
        print('JSON Error')
    time.sleep(10)
#     Again, the Sleep function is to keep Trulia from blocking me out of their website. I found less than 2 minutes to be problematic in 2020
    if i % 50 == 0:
        try:
            listing_details_csv = pd.DataFrame(listing_details, columns =['date','dateListed','description', 'full_address', 'formatted_address2', 'street_address', 'state', 'county', 'city', 'neighborhood', 'zipcd', 'list_photos', 'furnished','lat', 'lng', 'price', 'USD', 'typename', 'priceChange', 'bed', 'bath', 'sqft', 'LotSize', 'walk', 'bike', 'transit', 'Sidewalks', 'KidsPlayOutside', 'Wildlife', 'FriendlyNeighbors', 'GroceryWalk', 'LongTermResidents', 'WalkAloneNight', 'HolidaySpirit', 'travel_dist_to', 'travel_time_to', 'travel_dist_from', 'travel_time_from', 'listing_url', 'cover_image_url','listingID', 'maloneID', 'trailheads', 'parks', 'closest_park', 'closest_park_id', 'closest_park_rating', 'closest_park_ratings', 'closest_park_distance', 'most_popular_park', 'most_popular_park_id', 'most_popular_park_rating', 'most_popular_park_ratings', 'most_popular_park_distance', 'closest_trailhead', 'closest_trailhead_id', 'closest_trailhead_rating', 'closest_trailhead_ratings','closest_trailhead_distance', 'most_popular_trailhead', 'most_popular_trailhead_id', 'most_popular_trailhead_rating', 'most_popular_trailhead_ratings', 'most_popular_trailhead_distance', 'es_name', 'es_rating', 'es_max_rating', 'ms_name', 'ms_rating', 'ms_max_rating', 'hs_name', 'hs_rating', 'hs_max_rating', 'crime', 'Restaurants', 'Groceries', 'Nightlife'])
            listing_details_csv.to_csv('listing_details_csv_in_progress_draft.csv')
            print("Draft published")
        except:
            print("Draft publishing error")
            continue
    else:
        continue

code_c_ts = datetime.now(tz=None)
print("Completed at:",code_c_ts)
# print(listing_details)
listing_details_csv = pd.DataFrame(listing_details, columns =['date','dateListed','description', 'full_address', 'formatted_address2', 'street_address', 'state', 'county', 'city', 'neighborhood', 'zipcd', 'list_photos', 'furnished','lat', 'lng', 'price', 'USD', 'typename', 'priceChange', 'bed', 'bath', 'sqft', 'LotSize', 'walk', 'bike', 'transit', 'Sidewalks', 'KidsPlayOutside', 'Wildlife', 'FriendlyNeighbors', 'GroceryWalk', 'LongTermResidents', 'WalkAloneNight', 'HolidaySpirit', 'travel_dist_to', 'travel_time_to', 'travel_dist_from', 'travel_time_from', 'listing_url', 'cover_image_url','listingID', 'maloneID', 'trailheads', 'parks', 'closest_park', 'closest_park_id', 'closest_park_rating', 'closest_park_ratings', 'closest_park_distance', 'most_popular_park', 'most_popular_park_id', 'most_popular_park_rating', 'most_popular_park_ratings', 'most_popular_park_distance', 'closest_trailhead', 'closest_trailhead_id', 'closest_trailhead_rating', 'closest_trailhead_ratings','closest_trailhead_distance', 'most_popular_trailhead', 'most_popular_trailhead_id', 'most_popular_trailhead_rating', 'most_popular_trailhead_ratings', 'most_popular_trailhead_distance', 'es_name', 'es_rating', 'es_max_rating', 'ms_name', 'ms_rating', 'ms_max_rating', 'hs_name', 'hs_rating', 'hs_max_rating', 'crime', 'Restaurants', 'Groceries', 'Nightlife'])
listing_details_csv.to_csv('listing_details_csv_draft.csv')
print("Complete")

1  of  1  -  22:31:36.959810 https://www.trulia.com/p/ca/redwood-city/address-not-disclosed-redwood-city-ca-94061--2208275442
Available
{'url': '/p/ca/redwood-city/address-not-disclosed-redwood-city-ca-94061--2208275442', 'media': {'metaTagHeroImages': {'url': {'desktop': 'https://www.trulia.com/pictures/thumbs_6/zillowstatic/fp/2b7221f6d84a3b0e16c1d7453c7f475b-full.jpg', '__typename': 'MEDIA_ImageResource'}, '__typename': 'MEDIA_HeroImagePhoto'}, '__typename': 'HOME_Media', 'photos': [{'url': {'large': 'https://www.trulia.com/pictures/thumbs_5/zillowstatic/fp/2b7221f6d84a3b0e16c1d7453c7f475b-full.jpg', '__typename': 'MEDIA_ImageResource', 'thumbnail': 'https://www.trulia.com/pictures/thumbs_big/zillowstatic/fp/2b7221f6d84a3b0e16c1d7453c7f475b-full.jpg', 'extraSmallSrc': 'https://www.trulia.com/pictures/thumbs_4/zillowstatic/fp/2b7221f6d84a3b0e16c1d7453c7f475b-full.jpg', 'smallSrc': 'https://www.trulia.com/pictures/thumbs_4/zillowstatic/fp/2b7221f6d84a3b0e16c1d7453c7f475b-full.jpg', 'm

Completed at: 2022-05-25 22:31:47.686492
Complete
